# The Easiest Way to Deploy Your ML/DL Models in 2022: Streamlit + BentoML + DagsHub
## Deploy models as lightweight APIs with a user-friendly interface
![](images/pexels.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/photo/blue-red-and-black-abstract-painting-2130475/'>Steve Johnson</a>
    </strong>
</figcaption>


### Introduction

You have a ready machine learning model. Now what? What do you do with it? Do you keep it inside your Jupyter like a prized possession for no one to see? No, you make it as stupidly easy as possible for others to share and play around with your work. In other words, you *deploy* the model.

But how? Should you just share the model as a file? No, that would be the worst.

How about a Docker container? Yeah, that is better - the user will have it all to run your model locally. But... they would still need to do it inside a coding environment. Not very convenient. 

Then, what about an API? Well, good luck explaining what an API is to a non-programmer person. 

Hey, what if you built a web app with a minimal interface? Yes, this seems to be the best option. But how do build the app? Don't you need to learn a web framework for that? That's too much work!

### What problem are we solving?

### What tools will we be using to deploy?

### Step 1: Save the best model to BentoML local store

Let's start by importing the necessary libraries:

```python
import logging

import bentoml  # pip install bentoml --pre
import joblib
import tensorflow as tf
```

Make sure you install `bentoml` with the `--pre` tag since it is still in preview. 

Below, we will create a couple helper functions to create and train a Keras convolution model:

```python
def get_keras_conv2d():
    """A function to build an instance of a Keras conv2d model."""
    
    model = ...
    
    return model


def fit_keras_conv2d():
    """
    A function to train a Keras conv2d model.
    """
    model = get_keras_conv2d()
    
    #-- Fit model with early stopping and 30 epochs on the images --#
    
    return model
```

I've left the body of the first function but all it does is it creates a Conv2D instance with three hidden layers, with dropout and MaxPool layers in-between. We don't have to focus on model architecture too much. 

`fit_keras_conv2d` uses the first function and trains the obtained model with early stopping and 30 epochs.

Next, we create a function to save the model to BentoML local store:

```python
def save(model, bentoml_name, path):
    """
    A function to save a given model to BentoML local store and with joblib.
    """
    bentoml.keras.save(bentoml_name, model, store_as_json_and_weights=True)
    
    joblib.dump(model, path)
```

The `keras.save` function specifically saves Keras models in a format suitable for other BentoML operations. 

So, let's run these functions to get a ready model:

```python
def main():
    
    model = fit_keras_conv2d()
    
    logging.log(logging.INFO, "Saving...")
    
    save(model, 
         "keras_conv2d_smaller", 
         "models/keras_conv2d_smaller.joblib")
    
    logging.log(logging.INFO, "Done!")
    
if __name__ == "__main__":
    main()
```

After the training and saving is done, you can run the below command to get a list of models in the BentoML store:

```bash
$ bentoml models list
```
![](images/models_list.gif)


The saved models are officially called *tags* in the BentoML docs. By default, all models will be saved under your home directory and the `bentoml/models` folder with a random tag name, in case there are multiple models with the same name.

If you go into the given path, you will find files like these:

```
checkpoint
model.yaml
saved_model_json.json
saved_model_weights.data-00000-of-00001
saved_model_weights.index
```

You can always load the model back using the `load_runner` function preceded by the relevant framework name:

```python
model = bentoml.keras.load_runner("keras_conv2d_smaller:latest")

# Load a sample image from memory
img = ...

print(model.run(img))
```

After loading, the model can be used for prediction using its `run` method, which calls the `predict` method of Keras `Model` object under the hood.

### Step 2: Create the service

```python
import logging
import warnings

import numpy as np
import bentoml
from bentoml.io import Text, NumpyNdarray
from skimage.transform import resize

```

```python
def create_bento_service_keras(bento_name):
    """
    Create a Bento service for a Keras model.
    """
    # Load the model
    keras_model = bentoml.keras.load_runner(bento_name)

    # Create the service
    service = bentoml.Service(bento_name + "_service", runners=[keras_model])

    return keras_model, service
```

```python
model, service = create_bento_service_keras("conv2d_larger_dropout")


# Create an API function
@service.api(input=Text(), output=NumpyNdarray())
def predict(image_str) -> np.ndarray:
    """
    Predict pet pawpularity from an image using the given Bento.
    """
    # Convert the image back to numpy array
    image = np.fromstring(image_str, np.uint8)
    image = resize(image, (224, 224, 3))
    image = image / 255.0

    result = model.run(image)

    return result
```

```bash
$ bentoml serve service.py:service --reload
```

### Step 3: Build the bento

```bash
$ pip install pipreqs

$ pipreqs .
```

```
bentoml==1.0.0a7
catboost==0.26.1
dagshub==0.1.8
joblib==0.17.0
keras==2.8.0
lightgbm==2.3.1
matplotlib==3.3.1
mlflow==1.24.0
numpy==1.22.3
pandas==1.3.2
scikit_image==0.18.3
scikit_learn==1.0.2
seaborn==0.11.0
skimage==0.0
tensorflow==2.8.0
tqdm==4.50.0
xgboost==1.4.2

```

```YAML
service: "service.py:service"
include:
 - "service.py"
python:
  packages:
   - scikit_learn==1.0.2
   - numpy==1.22.3
   - tensorflow==2.8.0
   - scikit_image==0.18.3
```

```bash
$ bentoml build
```

```bash
$ bentoml list
```

### Step 4: Deploy to Heroku

```bash
    ├───apis
    │       openapi.yaml
    ├───env
    │   ├───conda
    │   ├───docker
    │   │       Dockerfile
    │   │       entrypoint.sh
    │   │       init.sh
    │   └───python
    │           requirements.lock.txt
    │           requirements.txt
    ├───models
    │   └───keras_conv2d
    │       │   latest
    │       │
    │       └───b52h7x5xpk2bejcl
    │               checkpoint
    │               model.yaml
    │               saved_model_json.json
    │               saved_model_weights.data-00000-of-00001
    │               saved_model_weights.index
    └───src
    │       service.py
    │   bento.yaml
    │   README.md
```

```bash
$ heroku login
```

```bash
$ heroku container:login
```

```bash
$ heroku create pet-pawpularity
```

```bash
$ cd ~/bentoml/bentos/keras_conv2d_smaller_service/uaaub3v3cku3ejcl
$ cd env/docker

$ heroku container:push web --app pet-pawpularity --context-path=../..
```

```bash
$ heroku container:release web --app pet-pawpularity
```

### Step 5: Build the UI with Streamlit

### Conclusion